In [1]:
import numpy as np
from pathlib import Path
import os

In [4]:
import imgaug as ia
from imgaug import augmenters as iaa
from PIL import Image
from scipy import misc
import shutil

# heavy augmentation

In [39]:
# Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
# e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second
# image.
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

# Define our sequence of augmentation steps that will be applied to every image.
seq = iaa.Sequential(
    [
        #
        # Apply the following augmenters to most images.
        #
#         iaa.Fliplr(0.5), # horizontally flip 50% of all images
#         iaa.Flipud(0.2), # vertically flip 20% of all images

        # crop some of the images by 0-10% of their height/width
        sometimes(iaa.Crop(percent=(0, 0.1))),

        # Apply affine transformations to some of the images
        # - scale to 80-120% of image height/width (each axis independently)
        # - translate by -20 to +20 relative to height/width (per axis)
        # - rotate by -45 to +45 degrees 
        # - shear by -16 to +16 degrees
        # - order: use nearest neighbour or bilinear interpolation (fast)
        # - mode: use any available mode to fill newly  created pixels
        #         see API or scikit-image for which modes are available
        # - cval: if the mode is constant, then use a random brightness
        #         for the newly created pixels (e.g. sometimes black,
        #         sometimes white)
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
            rotate=(-45, 45),
            shear=(-16, 16),
            order=[0, 1],
            cval=(0, 255),
            mode=ia.ALL
        )),

        #
        # Execute 0 to 5 of the following (less important) augmenters per
        # image. Don't execute all of them, as that would often be way too
        # strong.
        #
        iaa.SomeOf((0, 5),
            [
                # Convert some images into their superpixel representation,
                # sample between 20 and 200 superpixels per image, but do
                # not replace all superpixels with their average, only
                # some of them (p_replace).
#                 sometimes(
#                     iaa.Superpixels(
#                         p_replace=(0, 1.0),
#                         n_segments=(20, 200)
#                     )
#                 ),

                # Blur each image with varying strength using
                # gaussian blur (sigma between 0 and 3.0),
                # average/uniform blur (kernel size between 2x2 and 7x7)
                # median blur (kernel size between 3x3 and 11x11).
                iaa.OneOf([
                    iaa.GaussianBlur((0, 3.0)),
                    iaa.AverageBlur(k=(2, 7)),
                    iaa.MedianBlur(k=(3, 11)),
                ]),

                # Sharpen each image, overlay the result with the original
                # image using an alpha between 0 (no sharpening) and 1
                # (full sharpening effect).
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),

                # Same as sharpen, but for an embossing effect.
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),

                # Search in some images either for all edges or for
                # directed edges. These edges are then marked in a black
                # and white image and overlayed with the original image
                # using an alpha of 0 to 0.7.
                sometimes(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0, 0.7)),
                    iaa.DirectedEdgeDetect(
                        alpha=(0, 0.7), direction=(0.0, 1.0)
                    ),
                ])),

                # Add gaussian noise to some images.
                # In 50% of these cases, the noise is randomly sampled per
                # channel and pixel.
                # In the other 50% of all cases it is sampled once per
                # pixel (i.e. brightness change).
                iaa.AdditiveGaussianNoise(
                    loc=0, scale=(0.0, 0.05*255), per_channel=0.5
                ),

                # Either drop randomly 1 to 10% of all pixels (i.e. set
                # them to black) or drop them on an image with 2-5% percent
                # of the original size, leading to large dropped
                # rectangles.
                iaa.OneOf([
                    iaa.Dropout((0.01, 0.1), per_channel=0.5),
                    iaa.CoarseDropout(
                        (0.03, 0.15), size_percent=(0.02, 0.05),
                        per_channel=0.2
                    ),
                ]),

                # Invert each image's chanell with 5% probability.
                # This sets each pixel value v to 255-v.
                iaa.Invert(0.05, per_channel=True), # invert color channels

                # Add a value of -10 to 10 to each pixel.
                iaa.Add((-10, 10), per_channel=0.5),

                # Change brightness of images (50-150% of original value).
                iaa.Multiply((0.5, 1.5), per_channel=0.5),

                # Improve or worsen the contrast of images.
                iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5),

                # Convert each image to grayscale and then overlay the
                # result with the original with random alpha. I.e. remove
                # colors with varying strengths.
                iaa.Grayscale(alpha=(0.0, 1.0)),

                # In some images move pixels locally around (with random
                # strengths).
#                 sometimes(
#                     iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)
#                 ),

                # In some images distort local areas with varying strength.
#                 sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05)))
            ],
            # do all of the above augmentations in random order
            random_order=True
        )
    ],
    # do all of the above augmentations in random order
    random_order=True
)

Medium augmentation

In [6]:
def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

In [7]:
def save_image( npdata, outfilename ) :
    img = Image.fromarray( np.asarray( np.clip(npdata,0,255), dtype="uint8"), 'RGB' )
    img.save( outfilename,quality=33 )

In [8]:
def augmentation(img,target_path):
    images_aug = seq.augment_images(img)
    
    pad_name = len(os.listdir(target_path))
    [save_image(images_aug[i],(target_path/(str(pad_name+i)+'.jpeg'))) for i in range(0,len(img))]

In [43]:
CRUDE_TRAIN_PATH = Path("dataset/dataset/hero/train_ori")
TRAIN_PATH = Path("dataset/dataset/hero/train")
# VALID_PATH = Path("dataset/valid")
# TEST_PATH = Path("dataset/test")

plane_types = os.listdir(CRUDE_TRAIN_PATH)
aug_number = 1000

for plane in plane_types:
    # list all file in curde_dataset/plane
    imgs =  os.listdir(CRUDE_TRAIN_PATH/plane)
    print(plane)
    len_imgs = len(imgs)
    
    # split dataset into train, validation and test
    # with ratio 8:1:1
#     imgs_train = imgs[:int(len_imgs*0.8)]
#     imgs_val = imgs[int(len_imgs*0.8):int(len_imgs*0.9)]
#     imgs_test = imgs[int(len_imgs*0.9):]
    
    # craete directory target if doesnt exist
    if not os.path.exists(TRAIN_PATH/plane) : os.makedirs(TRAIN_PATH/plane)   
#     if not os.path.exists(VALID_PATH/plane) : os.makedirs(VALID_PATH/plane)
#     if not os.path.exists(TEST_PATH/plane) : os.makedirs(TEST_PATH/plane)
    
    # augmentation data 
    imgs = [load_image(CRUDE_TRAIN_PATH/plane/img) for img in imgs]
    for img in imgs:
        img_aug = np.array([img for i in range(0,aug_number)],dtype='uint8')[:,:,:,:3]
        augmentation(img_aug, TRAIN_PATH/plane)
        
#     # moving data validation
#     imgs = [img for img in imgs_val]
#     for img in imgs:
#         shutil.move(CRUDE_TRAIN_PATH/plane/img,VALID_PATH/plane/img)
        
#     # moving data test
#     imgs = [img for img in imgs_test]
#     for img in imgs:
#         shutil.move(CRUDE_TRAIN_PATH/plane/img,TEST_PATH/plane/img)
        
        

Phoenix
Crystal_Maiden
Shadow_Fiend
Sand_King
Venomancer
Necrophos
Ursa
Elder_Titan
Anti-Mage
Tidehunter
Witch_Doctor
Invoker
Batrider
Tiny
Death_Prophet
Lifestealer
Riki
Pangolier
Medusa
Phantom_Lancer
Grimstroke
Kunkka
Lone_Druid
Beastmaster
Sniper
Templar_Assassin
Queen_of_Pain
Enigma
Terrorblade
Puck
Broodmother
Chen
Disruptor
Storm_Spirit
Enchantress
Leshrac
Lina
Timbersaw
Axe
Brewmaster
Snapfire
Chaos_Knight
Ogre_Magi
Legion_Commander
Omniknight
Bane
Tinker
Jakiro
Treant_Protector
Winter_Wyvern
Mirana
Clinkz
Dragon_Knight
Bristleback
Zeus
Razor
Mars
Underlord
Spirit_Breaker
Bloodseeker
Dazzle
Abaddon
Phantom_Assassin
Undying
Shadow_Demon
Outworld_Devourer
Lycan
Doom
Night_Stalker
Ancient_Apparition
Windranger
Lich
Void_Spirit
Drow_Ranger
Spectre
Warlock
Tusk
Shadow_Shaman
Gyrocopter
Dark_Willow
Naga_Siren
Earthshaker
Sven
Pudge
Earth_Spirit
Nature's_Prophet
Dark_Seer
Meepo
Ember_Spirit
Magnus
Huskar
Techies
Keeper_of_the_Light
Skywrath_Mage
Luna
Lion
Centaur_Warrunner
Io
Slark
Tr

In [15]:
plane_types_

['A330',
 'Embraer Legacy 600',
 'Fokker 70',
 'Dornier 328',
 'DC-8',
 'Boeing 747',
 'Beechcraft 1900',
 'ATR-42',
 'Tu-154',
 'Fokker 100',
 'FA-18',
 'King Air',
 'Challenger 600',
 'ATR-72',
 'A340',
 'An-12',
 'EMB-120',
 'Boeing 777',
 'Metroliner',
 'A380',
 'Yak-42',
 'SR-20',
 'MD-80',
 'Global Express',
 'A320',
 'Saab 2000',
 'DH-82',
 'Saab 340',
 'Boeing 737',
 'BAE-125',
 'DC-10',
 'Cessna 208',
 'Cessna Citation',
 'Tu-134',
 'CRJ-700',
 'DR-400',
 'DHC-6',
 'CRJ-200',
 'C-47',
 'Falcon 2000',
 'Boeing 717',
 'Boeing 757',
 'PA-28',
 'F-16',
 'Embraer ERJ 145',
 'MD-90',
 'DHC-1',
 'Boeing 767',
 'Falcon 900',
 'Fokker 50',
 'Boeing 707',
 'BAE 146',
 'Tornado',
 'Il-76',
 'L-1011',
 'Embraer E-Jet',
 'A300',
 'Spitfire',
 'DC-9',
 'DC-6',
 'Hawk T1',
 'DC-3',
 'Cessna 172',
 'A310',
 'Boeing 727',
 'Dash 8',
 'C-130',
 'Eurofighter Typhoon',
 'Gulfstream',
 'MD-11']